# ***REGRESSION***

In [ ]:
!pip install catboost
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 7.9 MB/s eta 0:00:00


In [ ]:
from sklearn.impute import SimpleImputer
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_squared_error
import optuna
from sklearn.model_selection import train_test_split

In [ ]:
train = pd.read_csv('/content/train (6).csv', index_col='Id')
train.head(3)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500


In [ ]:
columns = list(train.columns)
categorical_features = []
for i in columns:
    if str(type(train[i].iloc[0])) == "<class 'str'>":
        categorical_features.append(i)

In [ ]:
missing_values_table = train.isna().sum().reset_index()
missing_values_table.columns = ['Column', 'Missing Values']
missing_values_table['% of Total Values'] = (missing_values_table['Missing Values'] / len(train)) * 100

print(missing_values_table['% of Total Values'])

0      0.000000
1      0.000000
2     17.739726
3      0.000000
4      0.000000
        ...    
75     0.000000
76     0.000000
77     0.000000
78     0.000000
79     0.000000
Name: % of Total Values, Length: 80, dtype: float64


In [ ]:
a = list(missing_values_table['% of Total Values'])
num = []
cat = []
drp = []
for i in range(len(columns)):
    if missing_values_table['% of Total Values'].iloc[i] > 0:
        if missing_values_table['% of Total Values'].iloc[i] > 40:
            drp.append(str(missing_values_table['Column'].iloc[i]))
        elif str(missing_values_table['Column'].iloc[i]) in categorical_features:
            print(missing_values_table['Column'].iloc[i], missing_values_table['% of Total Values'].iloc[i], "CATEGORICAL")
            cat.append(str(missing_values_table['Column'].iloc[i]))
        else:
            print(missing_values_table['Column'].iloc[i], missing_values_table['% of Total Values'].iloc[i])
            num.append(str(missing_values_table['Column'].iloc[i]))

print(drp)


LotFrontage 17.73972602739726
MasVnrArea 0.547945205479452
BsmtQual 2.5342465753424657 CATEGORICAL
BsmtCond 2.5342465753424657 CATEGORICAL
BsmtExposure 2.6027397260273974 CATEGORICAL
BsmtFinType1 2.5342465753424657 CATEGORICAL
BsmtFinType2 2.6027397260273974 CATEGORICAL
Electrical 0.0684931506849315 CATEGORICAL
GarageType 5.5479452054794525 CATEGORICAL
GarageYrBlt 5.5479452054794525
GarageFinish 5.5479452054794525 CATEGORICAL
GarageQual 5.5479452054794525 CATEGORICAL
GarageCond 5.5479452054794525 CATEGORICAL
['Alley', 'MasVnrType', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature']


In [ ]:
train = train.drop(drp, axis = 1)

imputer = SimpleImputer(strategy='mean')
train[num] = imputer.fit_transform(train[num])

imputer_cat = SimpleImputer(strategy='most_frequent')
train[cat] = imputer_cat.fit_transform(train[cat])

In [ ]:
y = train['SalePrice']
x = train.drop('SalePrice', axis =1)

train_x, val_x, train_y, val_y = train_test_split(x, y, test_size=0.2)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
973,120,RL,55.0,7892,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,4,2010,WD,Normal
805,20,RL,75.0,9000,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,6,2006,WD,Family
1190,60,RL,60.0,7500,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,60,0,0,0,0,0,6,2010,WD,Normal


In [ ]:
columns = list(train_x.columns)
categorical_features = []
for i in columns:
    if str(type(train[i].iloc[0])) == "<class 'str'>":
        categorical_features.append(i)

In [ ]:
def objective(trial):
    params = {
        "iterations": trial.suggest_int("iterations", 500, 1000),
        "depth": trial.suggest_int("depth", 3, 6),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1, 5, log=True),
        "loss_function": "RMSE",
        "eval_metric": "RMSE",
        "random_seed": 42,
        "verbose": 0
    }

    model = CatBoostRegressor(**params)
    model.fit(train_x, train_y, eval_set=(val_x, val_y), verbose=1000, cat_features=categorical_features)

    preds = model.predict(val_x)
    rmse = mean_squared_error(val_y, preds)
    rmse = np.sqrt(rmse)
    return rmse

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)  # Количество итераций

# 📌 6. Обучение модели с лучшими параметрами
best_params = study.best_params
best_model = CatBoostRegressor(**best_params)
best_model.fit(train_x, train_y, eval_set=(val_x, val_y), verbose=100, cat_features=categorical_features)

# 📌 7. Оценка модели
final_preds = best_model.predict(val_x)
rmse = mean_squared_error(val_y, final_preds)
rmse = np.sqrt(rmse)
print(f"Final RMSE: {rmse}")

[I 2025-02-22 13:31:59,003] A new study created in memory with name: no-name-58f52034-88e2-4d88-946d-5248e270279f


0:	learn: 78333.0649674	test: 79904.7958863	best: 79904.7958863 (0)	total: 27.3ms	remaining: 18s


[I 2025-02-22 13:32:11,019] Trial 0 finished with value: 27590.170657313705 and parameters: {'iterations': 660, 'depth': 5, 'learning_rate': 0.016357363220883117, 'l2_leaf_reg': 1.0467755164243835}. Best is trial 0 with value: 27590.170657313705.


659:	learn: 15502.3089534	test: 27590.1706573	best: 27590.1706573 (659)	total: 11.8s	remaining: 0us

bestTest = 27590.17066
bestIteration = 659

0:	learn: 78018.7607019	test: 79644.0190750	best: 79644.0190750 (0)	total: 19.2ms	remaining: 18.9s


[I 2025-02-22 13:32:31,044] Trial 1 finished with value: 28054.269444924557 and parameters: {'iterations': 983, 'depth': 6, 'learning_rate': 0.023902850062876548, 'l2_leaf_reg': 3.5310269432730204}. Best is trial 0 with value: 27590.170657313705.


...

593:	learn: 8708.7251480	test: 27745.5415055	best: 27731.2561681 (592)	total: 5.21s	remaining: 0us

bestTest = 27731.25617
bestIteration = 592

Shrink model to first 593 iterations.
0:	learn: 69632.6423041	test: 71491.3158993	best: 71491.3158993 (0)	total: 11.4ms	remaining: 8.13s


[I 2025-02-22 13:35:24,653] Trial 18 finished with value: 28562.629881846802 and parameters: {'iterations': 712, 'depth': 5, 'learning_rate': 0.21317542189811298, 'l2_leaf_reg': 1.9932455456134346}. Best is trial 6 with value: 25505.915021309767.


711:	learn: 3451.4434144	test: 28588.2419094	best: 28562.6298818 (662)	total: 8.09s	remaining: 0us

bestTest = 28562.62988
bestIteration = 662

Shrink model to first 663 iterations.
0:	learn: 76839.6083414	test: 78564.7956370	best: 78564.7956370 (0)	total: 15.2ms	remaining: 9.2s


[I 2025-02-22 13:35:36,463] Trial 19 finished with value: 27636.592120178866 and parameters: {'iterations': 608, 'depth': 6, 'learning_rate': 0.04833367065674872, 'l2_leaf_reg': 1.5504919469866252}. Best is trial 6 with value: 25505.915021309767.


607:	learn: 9007.5238814	test: 27636.5921202	best: 27636.5921202 (607)	total: 11.7s	remaining: 0us

bestTest = 27636.59212
bestIteration = 607

0:	learn: 75633.7775578	test: 77511.7476552	best: 77511.7476552 (0)	total: 14.1ms	remaining: 8.56s
100:	learn: 18169.0442353	test: 31046.3187900	best: 31046.3187900 (100)	total: 2.28s	remaining: 11.4s
200:	learn: 14158.4309190	test: 29124.2526013	best: 29124.2526013 (200)	total: 4.18s	remaining: 8.45s
300:	learn: 11582.0708109	test: 28465.6510796	best: 28465.6510796 (300)	total: 6.09s	remaining: 6.19s
400:	learn: 9875.7318011	test: 28048.5426547	best: 28043.2554774 (398)	total: 7.97s	remaining: 4.09s
500:	learn: 8681.9566210	test: 27841.4177588	best: 27840.8079547 (493)	total: 9.94s	remaining: 2.1s
600:	learn: 7654.3280918	test: 27619.5164463	best: 27609.0403144 (598)	total: 11.8s	remaining: 118ms
606:	learn: 7594.7749448	test: 27610.8376438	best: 27609.0403144 (598)	total: 11.9s	remaining: 0us

bestTest = 27609.04031
bestIteration = 598

Shrin

In [ ]:
print(best_params)

{'iterations': 607, 'depth': 6, 'learning_rate': 0.0677948134301802, 'l2_leaf_reg': 2.0814646273997335}


In [ ]:
test = pd.read_csv('/content/test (3).csv')
test.head(3)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal


In [ ]:
test = test.drop(drp, axis = 1)

In [ ]:
x = list(val_x.columns)
y = list(test.columns)
for i in y:
    if i not in x:
        print(i)

Id


In [ ]:
missing_values_table = test.isna().sum().reset_index()
missing_values_table.columns = ['Column', 'Missing Values']
missing_values_table['% of Total Values'] = (missing_values_table['Missing Values'] / len(test)) * 100

print(missing_values_table['% of Total Values'])

0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
     ... 
68    0.0
69    0.0
70    0.0
71    0.0
72    0.0
Name: % of Total Values, Length: 73, dtype: float64


In [ ]:
a = list(missing_values_table['% of Total Values'])
num = []
cat = []
drp = []
for i in range(len(columns)):
    if missing_values_table['% of Total Values'].iloc[i] > 0:
        if missing_values_table['% of Total Values'].iloc[i] > 40:
            drp.append(str(missing_values_table['Column'].iloc[i]))
        elif str(missing_values_table['Column'].iloc[i]) in categorical_features:
            print(missing_values_table['Column'].iloc[i], missing_values_table['% of Total Values'].iloc[i], "CATEGORICAL")
            cat.append(str(missing_values_table['Column'].iloc[i]))
        else:
            print(missing_values_table['Column'].iloc[i], missing_values_table['% of Total Values'].iloc[i])
            num.append(str(missing_values_table['Column'].iloc[i]))

MSZoning 0.2741603838245374 CATEGORICAL
LotFrontage 15.558601782042494
Utilities 0.1370801919122687 CATEGORICAL
Exterior1st 0.06854009595613435 CATEGORICAL
Exterior2nd 0.06854009595613435 CATEGORICAL
MasVnrArea 1.0281014393420151
BsmtQual 3.015764222069911 CATEGORICAL
BsmtCond 3.0843043180260454 CATEGORICAL
BsmtExposure 3.015764222069911 CATEGORICAL
BsmtFinType1 2.878684030157642 CATEGORICAL
BsmtFinSF1 0.06854009595613435
BsmtFinType2 2.878684030157642 CATEGORICAL
BsmtFinSF2 0.06854009595613435
BsmtUnfSF 0.06854009595613435
TotalBsmtSF 0.06854009595613435
BsmtFullBath 0.1370801919122687
BsmtHalfBath 0.1370801919122687
KitchenQual 0.06854009595613435 CATEGORICAL
Functional 0.1370801919122687 CATEGORICAL
GarageType 5.20904729266621 CATEGORICAL
GarageYrBlt 5.346127484578479
GarageFinish 5.346127484578479 CATEGORICAL
GarageCars 0.06854009595613435
GarageArea 0.06854009595613435
GarageQual 5.346127484578479 CATEGORICAL
GarageCond 5.346127484578479 CATEGORICAL
SaleType 0.06854009595613435 CA

In [ ]:
imputer = SimpleImputer(strategy='mean')
test[num] = imputer.fit_transform(test[num])

imputer_cat = SimpleImputer(strategy='most_frequent')
test[cat] = imputer_cat.fit_transform(test[cat])

In [ ]:
final_preds = best_model.predict(test)

In [ ]:
final = pd.DataFrame({
    "Id": test['Id'],
    "SalePrice": final_preds
})
final.head(3)

,Id,SalePrice
0,1461,116908.794116
1,1462,163613.449249
2,1463,184095.361491


In [ ]:
final.to_csv('final_0.csv', index=False)

# Classification

In [ ]:
!pip install optuna
!pip install catboost

In [ ]:
from sklearn.impute import SimpleImputer
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier, Pool
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
train = pd.read_csv('/content/train (7).csv', index_col=False)
train = train.drop('PassengerId', axis=1)
test = pd.read_csv('/content/test (4).csv', index_col=False)

In [ ]:
train.head(3)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [ ]:
missing_values_table = train.isna().sum().reset_index()
missing_values_table.columns = ['Column', 'Missing Values']
missing_values_table['% of Total Values'] = (missing_values_table['Missing Values'] / len(train)) * 100

print(missing_values_table['% of Total Values'])

0      0.000000
1      0.000000
2      0.000000
3      0.000000
4     19.865320
5      0.000000
6      0.000000
7      0.000000
8      0.000000
9     77.104377
10     0.224467
Name: % of Total Values, dtype: float64


In [ ]:
columns = list(train.columns)
categorical_features = []
for i in columns:
    if str(type(train[i].iloc[0])) == "<class 'str'>":
        categorical_features.append(i)

In [ ]:
a = list(missing_values_table['% of Total Values'])
num = []
cat = []
drp = []
for i in range(len(a)):
    if missing_values_table['% of Total Values'].iloc[i] > 0:
        if missing_values_table['% of Total Values'].iloc[i] > 40:
            drp.append(str(missing_values_table['Column'].iloc[i]))
        elif str(missing_values_table['Column'].iloc[i]) in categorical_features:
            print(missing_values_table['Column'].iloc[i], missing_values_table['% of Total Values'].iloc[i], "CATEGORICAL")
            cat.append(str(missing_values_table['Column'].iloc[i]))
        else:
            print(missing_values_table['Column'].iloc[i], missing_values_table['% of Total Values'].iloc[i])
            num.append(str(missing_values_table['Column'].iloc[i]))

print(drp)

Age 19.865319865319865
Embarked 0.22446689113355783 CATEGORICAL
['Cabin']


In [ ]:
train = train.drop(drp, axis = 1)

imputer = SimpleImputer(strategy='mean')
train[num] = imputer.fit_transform(train[num])

imputer_cat = SimpleImputer(strategy='most_frequent')
train[cat] = imputer_cat.fit_transform(train[cat])

In [ ]:
y = train['Survived']
x = train.drop('Survived', axis =1)

train_x, val_x, train_y, val_y = train_test_split(x, y, test_size=0.2)

In [ ]:
columns = list(train_x.columns)
categorical_features = []
for i in columns:
    if str(type(train[i].iloc[0])) == "<class 'str'>":
        categorical_features.append(i)

In [ ]:
def objective(trial):
    params = {
        "iterations": trial.suggest_int("iterations", 500, 2000),
        "depth": trial.suggest_int("depth", 4, 7),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1, 5, log=True),
        "loss_function": "Logloss",
        "eval_metric": "Accuracy",
        "random_seed": 42,
        "verbose": 0
    }

    model = CatBoostClassifier(**params)
    model.fit(train_x, train_y, eval_set=(val_x, val_y), verbose=1000, cat_features=categorical_features)

    preds = model.predict(val_x)
    accuracy = accuracy_score(val_y, preds)
    return accuracy

# 📌 5. Оптимизация гиперпараметров
study = optuna.create_study(direction="maximize")  # Для классификации — максимизируем accuracy
study.optimize(objective, n_trials=20)  # Количество итераций

# 📌 6. Обучение модели с лучшими параметрами
best_params = study.best_params
best_model = CatBoostClassifier(**best_params)
best_model.fit(train_x, train_y, eval_set=(val_x, val_y), verbose=100, cat_features=categorical_features)

# 📌 7. Оценка модели
final_preds = best_model.predict(val_x)
accuracy = accuracy_score(val_y, final_preds)
print(f"Final Accuracy: {accuracy:.4f}")


[I 2025-02-22 15:05:28,374] A new study created in memory with name: no-name-5696c2d7-b3d1-4ecd-9c50-1c3b744c4f02


0:	learn: 0.8146067	test: 0.7988827	best: 0.7988827 (0)	total: 2.92ms	remaining: 3.15s


[I 2025-02-22 15:05:30,557] Trial 0 finished with value: 0.8491620111731844 and parameters: {'iterations': 1078, 'depth': 5, 'learning_rate': 0.10281491066829387, 'l2_leaf_reg': 2.079064968064275}. Best is trial 0 with value: 0.8491620111731844.


1000:	learn: 0.9915730	test: 0.8212291	best: 0.8491620 (110)	total: 1.92s	remaining: 147ms
1077:	learn: 0.9929775	test: 0.8268156	best: 0.8491620 (110)	total: 2.06s	remaining: 0us

bestTest = 0.8491620112
bestIteration = 110

Shrink model to first 111 iterations.
0:	learn: 0.8188202	test: 0.7988827	best: 0.7988827 (0)	total: 4.02ms	remaining: 5.7s
1000:	learn: 0.9676966	test: 0.8268156	best: 0.8435754 (364)	total: 2.31s	remaining: 963ms


[I 2025-02-22 15:05:34,005] Trial 1 finished with value: 0.8435754189944135 and parameters: {'iterations': 1418, 'depth': 6, 'learning_rate': 0.043508919824835304, 'l2_leaf_reg': 3.6659717941077608}. Best is trial 0 with value: 0.8491620111731844.


...

1257:	learn: 0.9705056	test: 0.8324022	best: 0.8435754 (614)	total: 2.92s	remaining: 0us

bestTest = 0.843575419
bestIteration = 614

Shrink model to first 615 iterations.
0:	learn: 0.8188202	test: 0.7988827	best: 0.7988827 (0)	total: 3.62ms	remaining: 3.28s


[I 2025-02-22 15:06:22,587] Trial 18 finished with value: 0.8379888268156425 and parameters: {'iterations': 907, 'depth': 6, 'learning_rate': 0.12819035429978284, 'l2_leaf_reg': 4.983150443455822}. Best is trial 0 with value: 0.8491620111731844.


906:	learn: 0.9887640	test: 0.7988827	best: 0.8379888 (129)	total: 2.19s	remaining: 0us

bestTest = 0.8379888268
bestIteration = 129

Shrink model to first 130 iterations.
0:	learn: 0.8146067	test: 0.7988827	best: 0.7988827 (0)	total: 2.39ms	remaining: 1.74s


[I 2025-02-22 15:06:24,680] Trial 19 finished with value: 0.8435754189944135 and parameters: {'iterations': 728, 'depth': 5, 'learning_rate': 0.06324796994402315, 'l2_leaf_reg': 2.469288457865203}. Best is trial 0 with value: 0.8491620111731844.


727:	learn: 0.9691011	test: 0.8268156	best: 0.8435754 (408)	total: 1.96s	remaining: 0us

bestTest = 0.843575419
bestIteration = 408

Shrink model to first 409 iterations.
0:	learn: 0.6306689	test: 0.6331918	best: 0.6331918 (0)	total: 2.52ms	remaining: 2.72s
100:	learn: 0.2803246	test: 0.4433972	best: 0.4266044 (31)	total: 200ms	remaining: 1.93s
200:	learn: 0.2079957	test: 0.4501661	best: 0.4266044 (31)	total: 393ms	remaining: 1.71s
300:	learn: 0.1601585	test: 0.4684582	best: 0.4266044 (31)	total: 587ms	remaining: 1.51s
400:	learn: 0.1227384	test: 0.4862125	best: 0.4266044 (31)	total: 799ms	remaining: 1.35s
500:	learn: 0.1003636	test: 0.5040966	best: 0.4266044 (31)	total: 999ms	remaining: 1.15s
600:	learn: 0.0832366	test: 0.5282830	best: 0.4266044 (31)	total: 1.19s	remaining: 942ms
700:	learn: 0.0718905	test: 0.5492214	best: 0.4266044 (31)	total: 1.37s	remaining: 738ms
800:	learn: 0.0637023	test: 0.5543152	best: 0.4266044 (31)	total: 1.56s	remaining: 540ms
900:	learn: 0.0556375	test: 0.

In [ ]:
print(best_params)

{'iterations': 1078, 'depth': 5, 'learning_rate': 0.10281491066829387, 'l2_leaf_reg': 2.079064968064275}


In [ ]:
test.head(3)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q


In [ ]:
a = list(test.columns)
num = []
cat = []
for i in range(len(a)):
    if missing_values_table['% of Total Values'].iloc[i] > 0:
        if missing_values_table['% of Total Values'].iloc[i] > 40:
            continue
        elif str(missing_values_table['Column'].iloc[i]) in categorical_features:
            print(missing_values_table['Column'].iloc[i], missing_values_table['% of Total Values'].iloc[i], "CATEGORICAL")
            cat.append(str(missing_values_table['Column'].iloc[i]))
        else:
            print(missing_values_table['Column'].iloc[i], missing_values_table['% of Total Values'].iloc[i])
            num.append(str(missing_values_table['Column'].iloc[i]))


Age 19.865319865319865
Embarked 0.22446689113355783 CATEGORICAL


In [ ]:
test = test.drop(drp, axis = 1)

imputer = SimpleImputer(strategy='mean')
test[num] = imputer.fit_transform(test[num])

imputer_cat = SimpleImputer(strategy='most_frequent')
test[cat] = imputer_cat.fit_transform(test[cat])

In [ ]:
id = test['PassengerId']
test = test.drop('PassengerId', axis = 1)

In [ ]:
final_preds = best_model.predict(test)

final = pd.DataFrame({
    "PassengerId": id,
    "Survived": final_preds
})
final.head(3)

,PassengerId,Survived
0,892,0
1,893,1
2,894,0


In [ ]:
final.to_csv('final_2.csv', index=False)

# Classification YOLO

In [ ]:
import pandas as pd
import os
from ultralytics import YOLO

DATA_DIR = "path/to/dataset"
TEST_DIR = "path/to/test"
OUTPUT_CSV = "submission.csv"
# n - s - m - l - x
model = YOLO("yolov11x-cls.pt")

model.train(
    data=DATA_DIR,
    epochs=10,
    imgsz=640,
    batch=16,
    lr0=1e-3,
    momentum=0.9,
    augment=True
)

model_path = "runs/train/exp/weights/best.pt"
model = YOLO(model_path)

test_images = [f for f in os.listdir(TEST_DIR) if f.endswith((".jpg", ".png"))]

predictions = []

for img_name in test_images:
    img_path = os.path.join(TEST_DIR, img_name)
    results = model(img_path)

    predicted_class = results[0].probs.top1
    predictions.append((img_name, predicted_class))

df = pd.DataFrame(predictions, columns=["filename", "class"])
df.to_csv(OUTPUT_CSV, index=False)

print(f"✅ Предсказания сохранены в {OUTPUT_CSV}")


# Detection YOLO

In [ ]:
#Преобразование из csv в yaml
import os
import pandas as pd
from PIL import Image

images_dir = "path/to/images"
csv_file = "path/to/annotations.csv"
output_dir = "path/to/yolo_format"

annotations = pd.read_csv(csv_file)

def normalize_coordinates(x1, y1, x2, y2, img_width, img_height):
    x_center = (x1 + x2) / 2 / img_width
    y_center = (y1 + y2) / 2 / img_height
    width = (x2 - x1) / img_width
    height = (y2 - y1) / img_height
    return x_center, y_center, width, height

for img_name in annotations['filename'].unique():
    img_path = os.path.join(images_dir, img_name)
    img = Image.open(img_path)
    img_width, img_height = img.size

    txt_file = os.path.join(output_dir, img_name.replace('.jpg', '.txt').replace('.png', '.txt'))

    with open(txt_file, 'w') as f:
        img_annotations = annotations[annotations['filename'] == img_name]

        for _, row in img_annotations.iterrows():
            class_id = row['class']
            x_center, y_center, width, height = normalize_coordinates(
                row['x1'], row['y1'], row['x2'], row['y2'], img_width, img_height
            )
            f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")



In [ ]:
# dataset.yaml
# train: path/to/dataset/images/train  # Путь к изображениям для обучения
# val: path/to/dataset/images/val      # Путь к изображениям для валидации
# test: path/to/dataset/images/test    # Путь к изображениям для тестирования

# nc: 3  # Количество классов
# names: ['car', 'person', 'dog']  # Имена классов


In [ ]:
import os
import pandas as pd
from ultralytics import YOLO

# === 1. ПОДГОТОВКА ДАННЫХ ===
DATA_YAML = "path/to/dataset.yaml"  # Файл конфигурации датасета
TEST_DIR = "path/to/test"  # Папка с тестовыми изображениями
OUTPUT_CSV = "submission.csv"  # Файл для предсказаний

# === 2. ОБУЧЕНИЕ YOLOv11 ===
model = YOLO("yolov11n.pt")  # Используем Nano-версию для быстрого обучения

model.train(
    data=DATA_YAML,  # Путь к файлу .yaml с разметкой
    epochs=10,       # Количество эпох
    imgsz=640,       # Размер входного изображения
    batch=16,        # Размер батча
    lr0=1e-3,        # Начальный learning rate
    momentum=0.9,    # Стабильность градиентов
    augment=True     # Включаем встроенные аугментации
)

model_path = "runs/train/exp/weights/best.pt"
model = YOLO(model_path)

test_images = [f for f in os.listdir(TEST_DIR) if f.endswith((".jpg", ".png"))]

predictions = []

for img_name in test_images:
    img_path = os.path.join(TEST_DIR, img_name)
    results = model(img_path)

    for result in results:
        for box in result.boxes.data:
            x1, y1, x2, y2, conf, cls = box.tolist()
            predictions.append((img_name, int(cls), conf, x1, y1, x2, y2))

df = pd.DataFrame(predictions, columns=["filename", "class", "confidence", "x1", "y1", "x2", "y2"])
df.to_csv(OUTPUT_CSV, index=False)

# Segmentation YOLO

In [ ]:
import pandas as pd
import os
from PIL import Image

csv_path = 'annotations.csv'
images_dir = 'path/to/images'
output_dir = 'yolo_annotations'

# Создаем папку для сохранения аннотаций
os.makedirs(output_dir, exist_ok=True)

data = pd.read_csv(csv_path)

def convert_to_yolo_format(image_width, image_height, x_min, y_min, x_max, y_max):
    x_center = (x_min + x_max) / 2 / image_width
    y_center = (y_min + y_max) / 2 / image_height
    width = (x_max - x_min) / image_width
    height = (y_max - y_min) / image_height
    return x_center, y_center, width, height

for image_name in os.listdir(images_dir):
    image_path = os.path.join(images_dir, image_name)

    with Image.open(image_path) as img:
        image_width, image_height = img.size

    image_annotations = data[data['image_name'] == image_name]

    annotation_file = os.path.join(output_dir, os.path.splitext(image_name)[0] + '.txt')
    with open(annotation_file, 'w') as f:
        for _, row in image_annotations.iterrows():
            class_id = row['class_id']
            x_min, y_min, x_max, y_max = row['x_min'], row['y_min'], row['x_max'], row['y_max']

            x_center, y_center, width, height = convert_to_yolo_format(image_width, image_height, x_min, y_min, x_max, y_max)

            f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")


In [ ]:
# train: path/to/train/images
# val: path/to/val/images
# test: path/to/test/images

# nc: 2  # Количество классов
# names: ['class_0', 'class_1']  # Имена классов

In [ ]:
from ultralytics import YOLO
import os
import pandas as pd

def mask_to_rle(mask):
    mask = mask.astype(np.uint8)

    rle = mask_util.encode(np.asfortranarray(mask))

    rle_string = str(rle['counts'], encoding='utf-8')
    return rle_string

data_path = 'path/to/data.yaml'

model = YOLO('yolo11n-seg.pt')

model.train(data=data_path, epochs=50, imgsz=640, batch_size=16, augment=True)

model.save('yolov11_segmentation_model.pt')

test_path = 'path/to/test/images'
results = model.predict(test_path, save=True)

predictions = []
for image_path, pred in zip(results.files, results.pred):
    file_name = os.path.basename(image_path)

    masks = pred.masks

    if masks is not None:
        for idx, mask in enumerate(masks):
            rle_string = mask_to_rle(mask)
            predictions.append([file_name, idx, rle_string])

df = pd.DataFrame(predictions, columns=['file_name', 'object_id', 'rle'])
df.to_csv('final.csv', index=False)


# Pose Detection

In [ ]:
from ultralytics import YOLO
import os
import pandas as pd
import numpy as np
from PIL import Image

data_path = 'path/to/data.yaml'

model = YOLO('yolov11-pose.pt')

model.train(data=data_path, epochs=50, imgsz=640, batch_size=16)

test_path = 'path/to/test/images'
results = model.predict(test_path, save=True)

def extract_keypoints(pred):
    keypoints = []
    for i, person in enumerate(pred.keypoints):
        keypoints.append(list(person.xy))
    return keypoints

predictions = []
for image_path, pred in zip(results.files, results.pred):
    file_name = os.path.basename(image_path)

    keypoints = extract_keypoints(pred)

    if keypoints:
        for person_id, person_keypoints in enumerate(keypoints):
            # Сохраняем координаты точек для каждого человека
            for idx, (x, y) in enumerate(person_keypoints):
                predictions.append([file_name, person_id, idx, x, y])  # Добавляем все ключевые точки

df = pd.DataFrame(predictions, columns=['file_name', 'person_id', 'keypoint_id', 'x', 'y'])
df.to_csv('final.csv', index=False)


# RecSys Surprise

In [ ]:
!pip install scikit-surprise optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 5.5 MB/s eta 0:00:00


In [ ]:
import random
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

In [ ]:
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import cross_validate, GridSearchCV

# 📌 Загружаем MovieLens 100K
data = Dataset.load_builtin('ml-100k')

# 📌 Определяем параметры для Grid Search
param_grid = {
    'n_factors': [50, 100, 150],  # Количество латентных факторов
    'lr_all': [0.002, 0.005, 0.01],  # Скорость обучения
    'reg_all': [0.02, 0.05, 0.1]  # Регуляризация
}

# 📌 Запускаем Grid Search
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=5, n_jobs=-1)
gs.fit(data)

# 📌 Выводим лучшие параметры
print(f"📊 Лучший RMSE: {gs.best_score['rmse']:.4f}")
print(f"🔧 Лучшие параметры: {gs.best_params['rmse']}")

# 📌 Обучаем финальную модель с лучшими параметрами
best_model = SVD(**gs.best_params['rmse'])
cross_validate(best_model, data, cv=5, verbose=True)


📊 Лучший RMSE: 0.9181
🔧 Лучшие параметры: {'n_factors': 150, 'lr_all': 0.01, 'reg_all': 0.1}
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9226  0.9176  0.9012  0.9308  0.9212  0.9187  0.0098  
MAE (testset)     0.7307  0.7283  0.7118  0.7365  0.7283  0.7271  0.0082  
Fit time          1.19    1.48    1.15    1.17    1.13    1.22    0.13    
Test time         0.11    0.15    0.07    0.15    0.07    0.11    0.03    


{'test_rmse': array([0.92262018, 0.91755708, 0.90115132, 0.93077068, 0.92121232]),
 'test_mae': array([0.73069412, 0.72832189, 0.71182612, 0.7364805 , 0.72833285]),
 'fit_time': (1.185208797454834,
  1.483828067779541,
  1.1467053890228271,
  1.1749987602233887,
  1.1339025497436523),
 'test_time': (0.1124725341796875,
  0.15291881561279297,
  0.07221841812133789,
  0.14911413192749023,
  0.0736684799194336)}

In [ ]:
import optuna
import pandas as pd
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import cross_validate

# 📌 Загружаем MovieLens 100K
data = Dataset.load_builtin('ml-100k')

# 📌 Функция для оптимизации гиперпараметров
def objective(trial):
    sim_options = {
        "name": trial.suggest_categorical("name", ["cosine", "pearson", "msd"]),
        "user_based": trial.suggest_categorical("user_based", [True, False])
    }

    model = KNNBasic(k=trial.suggest_int("k", 10, 50), sim_options=sim_options)

    cv_results = cross_validate(model, data, measures=["rmse"], cv=3, verbose=False)
    return cv_results["test_rmse"].mean()

# 📌 Оптимизация гиперпараметров с Optuna
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30)

# 📌 Выводим лучшие параметры
best_params = study.best_params
print(f"📊 Лучший RMSE: {study.best_value:.4f}")
print(f"🔧 Лучшие параметры: {best_params}")

# 📌 Обучаем финальную модель с лучшими параметрами
final_sim_options = {"name": best_params["name"], "user_based": best_params["user_based"]}
best_model = KNNBasic(k=best_params["k"], sim_options=final_sim_options)

cross_validate(best_model, data, measures=["rmse"], cv=5, verbose=True)


[I 2025-02-24 16:53:10,905] A new study created in memory with name: no-name-4ee24999-736b-4631-a413-5080a373b7b0


Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.


[I 2025-02-24 16:53:30,955] Trial 0 finished with value: 1.0273098011901922 and parameters: {'name': 'pearson', 'user_based': True, 'k': 19}. Best is trial 0 with value: 1.0273098011901922.


Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.


[I 2025-02-24 16:53:40,559] Trial 1 finished with value: 1.040094723866775 and parameters: {'name': 'pearson', 'user_based': True, 'k': 11}. Best is trial 0 with value: 1.0273098011901922.


...

[I 2025-02-24 16:59:32,391] Trial 28 finished with value: 1.046721221705121 and parameters: {'name': 'pearson', 'user_based': False, 'k': 46}. Best is trial 12 with value: 0.9851113891510596.


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


[I 2025-02-24 16:59:43,336] Trial 29 finished with value: 0.9900902320723297 and parameters: {'name': 'msd', 'user_based': True, 'k': 40}. Best is trial 12 with value: 0.9851113891510596.


📊 Лучший RMSE: 0.9851
🔧 Лучшие параметры: {'name': 'msd', 'user_based': False, 'k': 40}
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9697  0.9694  0.9790  0.9778  0.9769  0.9746  0.0041  
Fit time          0.76    0.53    0.48    0.52    0.52    0.56    0.10    
Test time         2.75    2.70    2.53    3.01    2.62    2.72    0.16    


{'test_rmse': array([0.96973846, 0.96939912, 0.97902707, 0.97778443, 0.97694979]),
 'fit_time': (0.7602078914642334,
  0.5323970317840576,
  0.48438191413879395,
  0.5157642364501953,
  0.5216193199157715),
 'test_time': (2.753460168838501,
  2.6966280937194824,
  2.530104160308838,
  3.0083537101745605,
  2.6171016693115234)}


  \begin{array}{|c|c|c|c|c|c|c|c|}\hline\\ \\
  \mathcal{} & Movielens 100k & RMSE & MAE & Time  \\ \hline\\
  a & SVD & 0.934 & 0.737 & 0:00:06   \\ \hline\\ \\
  b & SVD++ (cache_ratings=False) & 0.919 & 0.721 & 0:01:39\\ \hline\\ \\
  c & SVD++ (cache_ratings=True) & 0.919 & 0.721 & 0:01:22  \\ \hline\\ \\
  d & NMF & 0.963 & 0.758 & 0:00:06 \\ \hline\\ \\
  e & Slope One & 0.946 & 0.743 & 0:00:09 \\ \hline\\ \\
  f & k-NN & 0.98 & 0.774 & 0:00:08 \\ \hline\\ \\
  g & Slope One & 0.946 & 0.743 & 0:00:09 \\ \hline\\ \\
  h & Centered k-NN & 0.951 & 0.749 & 0:00:09 \\ \hline\\ \\
  i & k-NN Baseline & 0.931 & 0.733 & 0:00:13 \\ \hline\\ \\
  j & Co-Clustering & 0.963 & 0.753 & 0:00:06 \\ \hline\\ \\
  k & Random & 1.518 & 1.219 & 0:00:01 \\ \hline\\ \\
  l & Baseline & 0.944 & 0.748 & 0:00:02 \\ \hline
  \end{array}




# RecSys Rectools

In [ ]:
!pip install rectools
!pip install rectools[all]
!pip install rectools[torch]

In [ ]:
'''По факту от пользователя требуется обычная таблица, где каждая строка отражает одно взаимодействие: в первом столбце id юзера, во втором — айтема,
 а в третьем — скор взаимодействия (например, купил/ не купил). Если есть данные по времени, их тоже можно добавить. '''

In [ ]:
import numpy as np
import pandas as pd

from implicit.nearest_neighbours import TFIDFRecommender

from rectools import Columns
from rectools.dataset import Dataset
from rectools.metrics import (
    Precision,
    NDCG,
    AvgRecPopularity,
    Intersection,
    HitRate,
    SufficientReco,
    DebiasConfig,
    IntraListDiversity,
    Serendipity,
    calc_metrics,
)
from rectools.metrics.distances import PairwiseHammingDistanceCalculator
from rectools.models import *

In [ ]:
ratings = pd.read_csv(
    "ratings.dat",
    sep="::",
    engine="python",  # Because of 2-chars separators
    header=None,
    names=[Columns.User, Columns.Item, Columns.Weight, Columns.Datetime],
)
print(ratings.shape)
ratings.head()

(1000209, 4)


,user_id,item_id,weight,datetime
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [ ]:
ratings["datetime"] = pd.to_datetime(ratings["datetime"] * 10 ** 9)
ratings["datetime"].min(), ratings["datetime"].max()

(Timestamp('2000-04-25 23:05:32'), Timestamp('2003-02-28 17:49:50'))

In [ ]:
movies = pd.read_csv(
    "movies.dat",
    sep="::",
    engine="python",  # Because of 2-chars separators
    header=None,
    names=[Columns.Item, "title", "genres"],
    encoding_errors="ignore",
)
print(movies.shape)
split_dt = pd.Timestamp("2003-02-01")
df_train = ratings.loc[ratings["datetime"] < split_dt]
df_test = ratings.loc[ratings["datetime"] >= split_dt]
dataset = Dataset.construct(df_train)

(3883, 3)


In [ ]:
from rectools.models import LightFMWrapperModel
from lightfm import LightFM

model = LightFMWrapperModel(
        # внутри модели указываем параметр no_components
        # это размезность эмбеддингов, которые выучит модель
        model=LightFM(no_components = 30)
        )

model.fit(dataset)
recos = model.recommend(
    users=ratings[Columns.User].unique(),
    dataset=dataset,
    k=5,
    filter_viewed=True,
)

serendipity = Serendipity(k=10)
precision = Precision(k=10, r_precision=True)  # r_precision means division by min(k, n_user_test_items)
ndcg = NDCG(k=10, log_base=3)

movies["genre"] = movies["genres"].str.split("|")
genre_exploded = movies[["item_id", "genre"]].set_index("item_id").explode("genre")
genre_dummies = pd.get_dummies(genre_exploded, prefix="", prefix_sep="").groupby("item_id").sum()

precision_value = precision.calc(reco=recos, interactions=df_test)
print(f"precision: {precision_value}")

catalog = df_train[Columns.Item].unique()

serendipity_value = serendipity.calc(
    reco=recos,
    interactions=df_test,
    prev_interactions=df_train,
    catalog=catalog
)
print("Serendipity: ", serendipity_value)

print("NDCG: ", ndcg.calc(reco=recos, interactions=df_test))

distance_calculator = PairwiseHammingDistanceCalculator(genre_dummies)
ild = IntraListDiversity(k=10, distance_calculator=distance_calculator)
print("ILD: ", ild.calc(reco=recos))

precision: 0.032996632996633
Serendipity:  4.163643706464913e-06
NDCG:  0.041591868823295505
ILD:  3.4595364238410595


In [ ]:
model = SASRecModel(
    session_max_len=20,
    loss="softmax",
    n_factors=64,
    n_blocks=1,
    n_heads=4,
    dropout_rate=0.2,
    lr=0.001,
    batch_size=128,
    epochs=6,
    verbose=1,
    deterministic=True,
)

model.fit(dataset)
recos = model.recommend(
    users=ratings[Columns.User].unique(),
    dataset=dataset,
    k=5,
    filter_viewed=True,
)

serendipity = Serendipity(k=10)
precision = Precision(k=10, r_precision=True)  # r_precision means division by min(k, n_user_test_items)
ndcg = NDCG(k=10, log_base=3)

movies["genre"] = movies["genres"].str.split("|")
genre_exploded = movies[["item_id", "genre"]].set_index("item_id").explode("genre")
genre_dummies = pd.get_dummies(genre_exploded, prefix="", prefix_sep="").groupby("item_id").sum()

precision_value = precision.calc(reco=recos, interactions=df_test)
print(f"precision: {precision_value}")

catalog = df_train[Columns.Item].unique()

serendipity_value = serendipity.calc(
    reco=recos,
    interactions=df_test,
    prev_interactions=df_train,
    catalog=catalog
)
print("Serendipity: ", serendipity_value)

print("NDCG: ", ndcg.calc(reco=recos, interactions=df_test))

distance_calculator = PairwiseHammingDistanceCalculator(genre_dummies)
ild = IntraListDiversity(k=10, distance_calculator=distance_calculator)
print("ILD: ", ild.calc(reco=recos))

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/dist-packages/rectools/dataset/identifiers.py:60: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  unq_values = pd.unique(values)
/usr/local/lib/python3.11/dist-packages/rectools/models/nn/item_net.py:129: UserWarning: Ignoring `CatFeaturesItemNet` block because dataset doesn't contain item features.
  warnings.warn(explanation)
/usr/local/lib/python3.11/dist-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  Expected `str` but got `tuple` with value `('rectools.models.nn.item...net.CatFeaturesItemNet')` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
/usr/l

Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=6` reached.


precision: 0.04781144781144782
Serendipity:  4.502274814780399e-05
NDCG:  0.03513844945033891
ILD:  3.287748344370861


In [ ]:
model = BERT4RecModel(
    mask_prob=0.15,  # specify probability of masking tokens
    deterministic=True,
)

model.fit(dataset)
recos = model.recommend(
    users=ratings[Columns.User].unique(),
    dataset=dataset,
    k=5,
    filter_viewed=True,
)

serendipity = Serendipity(k=10)
precision = Precision(k=10, r_precision=True)  # r_precision means division by min(k, n_user_test_items)
ndcg = NDCG(k=10, log_base=3)

movies["genre"] = movies["genres"].str.split("|")
genre_exploded = movies[["item_id", "genre"]].set_index("item_id").explode("genre")
genre_dummies = pd.get_dummies(genre_exploded, prefix="", prefix_sep="").groupby("item_id").sum()

precision_value = precision.calc(reco=recos, interactions=df_test)
print(f"precision: {precision_value}")

catalog = df_train[Columns.Item].unique()

serendipity_value = serendipity.calc(
    reco=recos,
    interactions=df_test,
    prev_interactions=df_train,
    catalog=catalog
)
print("Serendipity: ", serendipity_value)

print("NDCG: ", ndcg.calc(reco=recos, interactions=df_test))

distance_calculator = PairwiseHammingDistanceCalculator(genre_dummies)
ild = IntraListDiversity(k=10, distance_calculator=distance_calculator)
print("ILD: ", ild.calc(reco=recos))

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/dist-packages/rectools/dataset/identifiers.py:60: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  unq_values = pd.unique(values)
/usr/local/lib/python3.11/dist-packages/rectools/models/nn/item_net.py:129: UserWarning: Ignoring `CatFeaturesItemNet` block because dataset doesn't contain item features.
  warnings.warn(explanation)
/usr/local/lib/python3.11/dist-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  Expected `str` but got `tuple` with value `('rectools.models.nn.item...net.CatFeaturesItemNet')` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
/usr/l

precision: 0.02630270963604297
Serendipity:  1.1552771056779242e-05
NDCG:  0.03232912926419424
ILD:  3.031059602649006



  \begin{array}{|c|c|c|c|c|c|c|c|}\hline\\ \\
  \mathcal{} & Movielens 1m & Precision & Serendipity & NDCG & ILD  \\ \hline\\
  a & SASRec (6 epochs) & 0.04781 & 4.5023e-05 & 0.03513 & 3.2877   \\ \hline\\ \\
  b & BERT4Rec & 0.02630 & 1.1553e-05 & 0.03233 & 3.0311 \\ \hline\\ \\
  c & implicit ALS Wrapper & 0.0515 & 5.4056e-05 & 0.05165 & 2.8392 \\ \hline\\ \\
  d & implicit BPR-MF Wrapper & 0.02997 & 3.19868e-06 & 0.03615 & 3.86506 \\ \hline\\ \\
  e & implicit ItemKNN Wrapper & 0.0456 & 3.0093e-05 & 0.04615 & 3.1726 \\ \hline\\ \\
  f & LightFM Wrapper & 0.05858 & 4.7578e-06 & 0.0604 & 3.6395 \\ \hline\\ \\
  g & EASE & 0.0367 & 3.0522e-05 & 0.03431 & 2.8200 \\ \hline\\ \\
  h & PureSVD & 0.05952 & 2.5205e-05 & 0.05248 & 3.0020 \\ \hline\\ \\
  j & Popular & 0.0330 & 4.1636e-06 & 0.04160 & 3.4595 \\ \hline\\ \\
  l & Random & 0.0131 & 1.6940e-05 & 0.00487 & 2.6273 \\ \hline
  \end{array}

# Детекция ботов

In [ ]:
import numpy as np
import pandas as pd

from implicit.nearest_neighbours import TFIDFRecommender

from rectools import Columns
from rectools.dataset import Dataset
from rectools.metrics import (
    Precision,
    NDCG,
    IntraListDiversity,
    Serendipity,
    calc_metrics,
)
from rectools.metrics.distances import PairwiseHammingDistanceCalculator
from rectools.models import LightFMWrapperModel
from lightfm import LightFM

# 1️⃣ Загрузка данных
ratings = pd.read_csv(
    "ratings.dat",
    sep="::",
    engine="python",
    header=None,
    names=[Columns.User, Columns.Item, Columns.Weight, Columns.Datetime],
)

ratings["datetime"] = pd.to_datetime(ratings["datetime"] * 10 ** 9)

movies = pd.read_csv(
    "movies.dat",
    sep="::",
    engine="python",
    header=None,
    names=[Columns.Item, "title", "genres"],
    encoding_errors="ignore",
)

# 2️⃣ ДЕТЕКЦИЯ БОТОВ

# 2.1 Определение аномальной активности пользователей
user_activity = ratings.groupby(Columns.User).agg(
    num_ratings=(Columns.Item, "count"),  # Количество оценок
    unique_movies=(Columns.Item, "nunique"),  # Уникальные фильмы
    rating_std=(Columns.Weight, "std"),  # Разброс оценок
    first_rating_time=("datetime", "min"),
    last_rating_time=("datetime", "max")
)

# 2.2 Добавляем скорость оценивания (оценки в день)
user_activity["rating_speed"] = user_activity["num_ratings"] / (
    (user_activity["last_rating_time"] - user_activity["first_rating_time"]).dt.days + 1
)

# 2.3 Удаление ботов по критериям:
thresholds = {
    "num_ratings": user_activity["num_ratings"].quantile(0.99),  # > 99-го перцентиля
    "unique_movies": user_activity["unique_movies"].quantile(0.01),  # < 1-го перцентиля
    "rating_std": 0.1,  # Нет разброса в оценках
    "rating_speed": user_activity["rating_speed"].quantile(0.99),  # > 99-го перцентиля
}

bots = user_activity[
    (user_activity["num_ratings"] > thresholds["num_ratings"]) |
    (user_activity["unique_movies"] < thresholds["unique_movies"]) |
    (user_activity["rating_std"] < thresholds["rating_std"]) |
    (user_activity["rating_speed"] > thresholds["rating_speed"])
]

# Удаляем ботов
ratings_cleaned = ratings[~ratings[Columns.User].isin(bots.index)]
print(f"Удалено {len(bots)} подозрительных пользователей.")

# 3️⃣ Деление на train/test
split_dt = pd.Timestamp("2003-02-01")
df_train = ratings_cleaned.loc[ratings_cleaned["datetime"] < split_dt]
df_test = ratings_cleaned.loc[ratings_cleaned["datetime"] >= split_dt]

# 4️⃣ Обучение LightFM
dataset = Dataset.construct(df_train)

model = LightFMWrapperModel(
    model=LightFM(no_components=30)
)
model.fit(dataset)

# 5️⃣ Генерация рекомендаций
recos = model.recommend(
    users=df_test[Columns.User].unique(),
    dataset=dataset,
    k=5,
    filter_viewed=True,
)

# 6️⃣ Оценка метрик
serendipity = Serendipity(k=10)
precision = Precision(k=10, r_precision=True)
ndcg = NDCG(k=10, log_base=3)

movies["genre"] = movies["genres"].str.split("|")
genre_exploded = movies[[Columns.Item, "genre"]].set_index(Columns.Item).explode("genre")
genre_dummies = pd.get_dummies(genre_exploded, prefix="", prefix_sep="").groupby(Columns.Item).sum()

precision_value = precision.calc(reco=recos, interactions=df_test)
print(f"precision: {precision_value}")

catalog = df_train[Columns.Item].unique()
serendipity_value = serendipity.calc(
    reco=recos,
    interactions=df_test,
    prev_interactions=df_train,
    catalog=catalog
)
print("Serendipity: ", serendipity_value)

print("NDCG: ", ndcg.calc(reco=recos, interactions=df_test))

distance_calculator = PairwiseHammingDistanceCalculator(genre_dummies)
ild = IntraListDiversity(k=10, distance_calculator=distance_calculator)
print("ILD: ", ild.calc(reco=recos))


# NLP NIR Natasha

In [ ]:
!pip install natasha

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 23.2 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=1c63601457de054370a39a15b73fae84c828da1412d67838ceaca15fd87d8aec
  Stored in directory: /root/.cache/pip/wheels/1a/b0/8c/4b75c4116c31f83c8f9f047231251e13cc74481cca4a78a9ce
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26097 sha256=4f8268b1c102f6a686fac1ad2b6f5e31d2e0c2c5f6fe16af2f6177850adac975
  Stored in directory: /root/.cache/pip/wheels/31/d7/d9/eec6891f78cac19a693bd40ecb8365d2f4613318c145ec9816
Successfully built docopt intervaltree


In [ ]:
import pandas as pd
from natasha import (
    Doc,
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
)

# Инициализация компонентов Natasha
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

# Функция для обработки предложения и извлечения сущностей
def extract_entities(sentence, sentence_id):
    doc = Doc(sentence)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.tag_ner(ner_tagger)

    entities = []
    for span in doc.spans:
        entities.append({
            'sentence_id': sentence_id,
            'entity': span.text,
            'entity_type': span.type
        })
    return entities

# Пример предложения
sentence = "Бурятия и Забайкальский край переданы из Сибирского федерального округа (СФО) в состав Дальневосточного (ДФО). Соответствующий указ подписал президент Владимир Путин, документ опубликован на официальном интернет-портале правовой информации. Этим же указом глава государства поручил руководителю своей администрации утвердить структуру и штатную численность аппаратов полномочных представителей президента в этих двух округах. После исключения Бурятии и Забайкалья в составе СФО остались десять регионов: Алтай, Алтайский край, Иркутская, Кемеровская, Новосибирская, Омская и Томская области, Красноярский край, Тува и Хакасия. Действующим полпредом президента в этом округе является бывший губернатор Севастополя, экс-заместитель командующего Черноморским флотом России Сергей Меняйло. В составе ДФО отныне 11 субъектов. Помимо Бурятии и Забайкалья, это Камчатский, Приморский и Хабаровский края, Амурская, Еврейская автономная, Магаданская и Сахалинская области, а также Якутия и Чукотка. Дальневосточное полпредство возглавляет Юрий Трутнев, совмещающий эту должность с постом вице-премьера в правительстве России. Федеральные округа были созданы в мае 2000 года в соответствии с указом президента Путина."
sentence_id = 1  # ID предложения

# Извлечение сущностей
entities = extract_entities(sentence, sentence_id)

# Создание DataFrame
df = pd.DataFrame(entities)

# Вывод результата
df.head(20)

,sentence_id,entity,entity_type
0,1,Бурятия,LOC
1,1,Забайкальский край,LOC
2,1,Сибирского федерального округа (СФО),LOC
3,1,Дальневосточного (ДФО),LOC
4,1,Владимир Путин,PER
5,1,Бурятии,LOC
6,1,Забайкалья,LOC
7,1,СФО,LOC
8,1,Алтай,LOC
9,1,Алтайский край,LOC


# Обработка + Word2Vec + TF-IDF

In [ ]:
!pip install pymorphy3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 45.2 MB/s eta 0:00:00


In [ ]:
categories = {
    "Жизнь человека": {
        "description": "Задания о ценности человеческой жизни, донорстве, осведомленности о заболеваниях",
        "examples": [
            "организация донорской акции",
            "переливание крови",
            "медицинское просвещение",
            "профилактика заболеваний",
            "первая помощь"
        ]
    },
    "Достоинство человека": {
        "description": "Задания об уважении к профессиям и людям разных социальных статусов",
        "examples": [
            "мастер-класс о уважении",
            "социальный статус",
            "профессиональная этика",
            "равенство возможностей",
            "толерантность",
            ""
        ]
    },
    "Права и свободы человека": {
        "description": "Изучение и защита прав человека",
        "examples": [
            "тест о правах человека",
            "конвенция о правах ребенка",
            "правовая грамотность",
            "защита свобод",
            "гражданские права"
        ]
    },
    "Патриотизм": {
        "description": "История государства, предки, патриотические организации",
        "examples": [
            "Россия",
            "Российская федерация",
            "СССР"
            "посещение военного музея",
            "волонтерство 9 мая",
            "история великой отечественной",
            "патриотический флешмоб",
            "герои россии"
        ]
    },
    "Гражданственность": {
        "description": "Процветание общества через личное участие",
        "examples": [
            "субботник",
            "экологическая акция",
            "гражданские инициативы",
            "благоустройство города",
            "общественный контроль"
        ]
    },
    "Служение Отечеству и ответственность за его судьбу": {
        "description": "История отечества и физическая подготовка граждан",
        "examples": [
            "военно-спортивные игры",
            "уроки мужества",
            "здоровье нации",
            "историческая реконструкция",
            "вахта памяти"
        ]
    },
    "Высокие нравственные идеалы": {
        "description": "Культура, идеи и творчество, формирующие мораль",
        "examples": [
            "обсуждение классической литературы",
            "нравственные дилеммы",
            "этические нормы",
            "моральный выбор",
            "духовные ценности",
            "история",
            "страны"
        ]
    },
    "Крепкая семья": {
        "description": "Совместная семейная деятельность и отношения",
        "examples": [
            "семейный квест",
            "генеалогическое древо",
            "совместный пикник",
            "семейные традиции",
            "родительский день"
        ]
    },
    "Созидательный труд": {
        "description": "Обучение практическим навыкам и физическая помощь",
        "examples": [
            "мастер-класс по ремеслам",
            "помощь пожилым",
            "трудовой десант",
            "профессиональные пробы",
            "социальный труд"
        ]
    },
    "Приоритет духовного над материальным": {
        "description": "Помощь нуждающимся вместо материальных благ",
        "examples": [
            "благотворительная акция",
            "помощь приюту",
            "духовные практики",
            "волонтерство в храме",
            "нематериальные ценности"
        ]
    },
    "Гуманизм": {
        "description": "Помощь людям и добрые дела",
        "examples": [
            "волонтерство в приюте",
            "помощь бездомным",
            "добрые письма",
            "поддержка инвалидов",
            "человеколюбие"
        ]
    },
    "Милосердие": {
        "description": "Помощь окружающим и информирование о болезнях",
        "examples": [
            "сбор вещей нуждающимся",
            "помощь пожилым",
            "донорство органов",
            "паллиативная помощь",
            "социальная поддержка"
        ]
    },
    "Справедливость": {
        "description": "Обсуждение и укрепление справедливости",
        "examples": [
            "дебаты о равенстве",
            "правовые квесты",
            "честное распределение",
            "борьба с дискриминацией",
            "равные возможности"
        ]
    },
    "Коллективизм": {
        "description": "Командная работа и взаимодействие в группе",
        "examples": [
            "групповой турпоход",
            "командный квиз",
            "совместный проект",
            "работа в команде",
            "коллективное решение"
        ]
    },
    "Взаимопомощь и взаимоуважение": {
        "description": "Волонтерство и поддержка окружающих",
        "examples": [
            "помощь одноклассникам",
            "тьюторство",
            "шефская помощь",
            "социальное наставничество",
            "взаимная поддержка"
        ]
    },
    "Историческая память и преемственность поколений": {
        "description": "История семьи и страны, связь поколений",
        "examples": [
            "интервью с ветеранами",
            "семейная летопись",
            "музей истории",
            "реконструкция событий",
            "устная история"
        ]
    },
    "Единство народов России": {
        "description": "Культурное многообразие и национальные традиции",
        "examples": [
            "фестиваль культур",
            "этнографический музей",
            "национальные ремесла",
            "межнациональный диалог",
            "традиционные обычаи"
        ]
    }
}

In [ ]:
import pandas as pd
import numpy as np
import re
from string import punctuation
import pymorphy3
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

morph = pymorphy3.MorphAnalyzer()
russian_stopwords = stopwords.words('russian')

def preprocess(text):
    text = text.lower()
    text = re.sub(f'[{punctuation}»«–…№©™•°]', '', text)
    words = re.findall(r'\b[a-zа-яё]+\b', text)
    lemmas = []
    for word in words:
        if word not in russian_stopwords and len(word) > 2:
            lemma = morph.parse(word)[0].normal_form
            lemmas.append(lemma)
    return ' '.join(lemmas)

# Создание корпуса документов
category_docs = []
for cat, data_s in categories.items():
    doc = f"{cat} {data_s['description']} {' '.join(data_s['examples'])}"
    category_docs.append(doc)

# Инициализация TF-IDF
vectorizer = TfidfVectorizer(tokenizer=preprocess)
tfidf_matrix = vectorizer.fit_transform(category_docs)

def classify_text(text):
    processed_text = preprocess(text)

    text_vector = vectorizer.transform([processed_text])
    print(text_vector)
    similarities = cosine_similarity(text_vector, tfidf_matrix)
    print(similarities)
    best_match_idx = similarities.argmax()
    best_category = list(categories.keys())[best_match_idx]
    accuracy = similarities[0][best_match_idx]


    return best_category, accuracy

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
input_text = "Организация группового турпохода с одноклассниками и работа в команде"
result = classify_text(input_text)
result

ValueError: setting an array element with a sequence.

# Определение похожих запросов

In [ ]:
import torch
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# ✅ Загружаем предобученную модель, поддерживающую русский язык
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

# 📌 Пример запросов
queries = [
    "Как приготовить борщ?",
    "Рецепт вкусного борща",
    "Где найти рецепт лазаньи?",
    "Как сварить суп?"
]

# 🔥 Получаем эмбеддинги
query_embeddings = model.encode(queries, convert_to_tensor=True)

# 🔥 Вычисляем матрицу косинусного сходства
similarity_matrix = cosine_similarity(query_embeddings.cpu().numpy())

# 📌 Выводим результаты
df = pd.DataFrame(similarity_matrix, index=queries, columns=queries)
print(df)


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.13k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

                           Как приготовить борщ?  Рецепт вкусного борща  \
Как приготовить борщ?                   1.000000               0.809563   
Рецепт вкусного борща                   0.809563               1.000000   
Где найти рецепт лазаньи?               0.617756               0.644937   
Как сварить суп?                        0.768549               0.594278   

                           Где найти рецепт лазаньи?  Как сварить суп?  
Как приготовить борщ?                       0.617756          0.768549  
Рецепт вкусного борща                       0.644937          0.594278  
Где найти рецепт лазаньи?                   1.000000          0.557159  
Как сварить суп?                            0.557159          1.000000  


In [ ]:
df.head()

,Как приготовить борщ?,Рецепт вкусного борща,Где найти рецепт лазаньи?,Как сварить суп?
Как приготовить борщ?,1.000000,0.809563,0.617756,0.768549
Рецепт вкусного борща,0.809563,1.000000,0.644937,0.594278
Где найти рецепт лазаньи?,0.617756,0.644937,1.000000,0.557159
Как сварить суп?,0.768549,0.594278,0.557159,1.000000


# rubert (semi-final version; sentiment analysys/text classification)

In [ ]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [ ]:
train=pd.read_csv(r'C:\VisualCode\T1\tr.csv', engine='python', encoding='utf-8', on_bad_lines="skip")
test=pd.read_csv(r'C:\VisualCode\T1\ts.csv', engine='python', encoding='utf-8', on_bad_lines = "skip")

# Разделение данных на признаки и метки
X_train, X_val, y_train, y_val = train_test_split(train['review'], train['sentiment'], test_size=0.2, random_state=42)

In [ ]:
class ReviewsDataset(Dataset):
    def __init__(self, reviews, labels=None, tokenizer=None, max_len=128):
        self.reviews = reviews
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, idx):
        review = self.reviews.iloc[idx]
        inputs = self.tokenizer.encode_plus(
            review,
            add_special_tokens=True,
            max_length=self.max_len,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )
        item = {key: val.squeeze(0) for key, val in inputs.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels.iloc[idx], dtype=torch.long)
        return item


In [ ]:
tokenizer = BertTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
model = BertForSequenceClassification.from_pretrained(
    "DeepPavlov/rubert-base-cased",
    num_labels=3  # У нас три класса: 0, 1, 2
)

In [ ]:
train_dataset = ReviewsDataset(X_train, y_train, tokenizer)
val_dataset = ReviewsDataset(X_val, y_val, tokenizer)
test_dataset = ReviewsDataset(test['review'], tokenizer=tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)

In [ ]:
from transformers import AdamW

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)

# Функция обучения
def train_epoch(model, data_loader, optimizer, device):
    model.train()
    total_loss = 0
    for batch in data_loader:
        inputs = {key: val.to(device) for key, val in batch.items() if key != 'labels'}
        labels = batch['labels'].to(device)

        outputs = model(**inputs, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    return total_loss / len(data_loader)

In [ ]:
import time

epochs = 3  # Можно увеличить для лучшего результата

for epoch in range(epochs):
    start_time = time.time()

    train_loss = train_epoch(model, train_loader, optimizer, device)

    end_time = time.time()
    epoch_duration = end_time - start_time

    hours = int(epoch_duration // 3600)
    minutes = int((epoch_duration % 3600) // 60)
    seconds = int(epoch_duration % 60)

    torch.save(model.state_dict(), f"model_epoch_{epoch+1}.pth")
    print(f"Эпоха {epoch + 1}/{3} завершена.")
    print(f"Потеря: {train_loss:.4f}, Время выполнения: {epoch_duration:.2f} секунд.")


In [ ]:
def predict(model, data_loader, device):
    model.eval()
    predictions = []
    with torch.no_grad():
        for batch in data_loader:
            inputs = {key: val.to(device) for key, val in batch.items()}
            outputs = model(**inputs)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            predictions.extend(preds)
    return predictions

# Предсказания
test['sentiment'] = predict(model, test_loader, device)


In [ ]:
ans = test[['index', 'sentiment']]
ans.head(3)
ans.to_csv('poputka_0.csv', index = False)

# RuBert + Distilbert (for classification and sentiment analysys)

In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from datasets import load_dataset
from transformers import Trainer, TrainingArguments
import torch

# Загрузка данных
train_dataset = load_dataset('csv', data_files='train.csv')['train']
test_dataset = load_dataset('csv', data_files='test.csv')['test']

# Токенизация
tokenizer = RobertaTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')

def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Загрузка модели для классификации текста
model = RobertaForSequenceClassification.from_pretrained('DeepPavlov/rubert-base-cased', num_labels=3)  # например, 3 класса

# Аргументы для тренировки
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    logging_dir='./logs',
)

# Создаем Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Обучаем модель
trainer.train()

# Получаем предсказания
predictions = trainer.predict(test_dataset)
pred_labels = torch.argmax(predictions.predictions, axis=1)
print(pred_labels)


In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
import torch

# Загружаем датасет
train_dataset = load_dataset('csv', data_files='train.csv')['train']
test_dataset = load_dataset('csv', data_files='test.csv')['test']

# Токенизация
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Загружаем модель для классификации текста
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3)  # 3 класса для примера

# Аргументы для тренировки
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    logging_dir='./logs',
)

# Создаем Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Обучаем модель
trainer.train()

# Получаем предсказания
predictions = trainer.predict(test_dataset)
pred_labels = torch.argmax(predictions.predictions, axis=1)
print(pred_labels)
